In [2]:
import pandas as pd
import numpy as np
from IPython import display as ICD


from pickle import load
from scripts.preprocess import load_index_data, load_tbills_data
from scripts.utils import print_results

In [13]:
def get_bootstrap_results(bootstrap):
    implied, params = {}, {}
    for n, res in enumerate(bootstrap.values()):
        start, end, pred = res.values()
        log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
        log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

        implied[n], params[n] = print_results(pred, log_mk, log_rf, False)
    
    implied = pd.concat(implied, axis=0)
    params = pd.concat(params, axis=0)
    sd = implied.groupby(level=1).std()
    
    implied = implied.groupby(level=1).mean()
    params = params.groupby(level=1).mean()
    
    implied.loc['std', ['alpha (%)', 'beta']] = sd.loc['value', ['alpha (%)', 'beta']].values
    return implied.reindex(index=['value', 'std']), params.reindex(index=['value', 'std'])

        
def gather_results(result_list):
    results = {}
    for name, path in result_list.items():
        if 'bootstrap' in path:
            bootstrap = load(open('../results/res_security_from_2010_bootstrap_^IXIC.pkl', 'rb'))
            implied, params = get_bootstrap_results(bootstrap)
        else:
            res = load(open(path, 'rb'))
            start, end, pred = res.values()
            log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
            log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values
            implied, params = print_results(pred, log_mk, log_rf, False)
            
        chunk = pd.concat([params, implied], axis=1)
        chunk.loc['value'] = chunk.loc['value'].apply(lambda x: f"{x:.2f}")
        chunk.loc['std'] = chunk.loc['std'].apply(lambda x: f"({x:.2f})")
        
        results[name] = chunk
    
    return pd.concat(results, axis=0)

# Full

## Boostrap

## Cochrane

In [6]:
res = load(open('../results/res_cochrane.pkl', 'rb'))
start, end = '1987-01-01', '2000-08-02'
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', test=True, prefix='../../').values

implied, params = print_results(res, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)


# Cochrane
# -2.12  0.96   99.23   16.04  0.76   4.76   1.26
# 1.09   0.09   1.75    0.79   0.02   0.13   0.9

Using parameters (annualized percentages)
E[log Rf]=5.26%, E[log Rm]=15.92%, V[log Rm]=14.80%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,34.308917,90.608776,71.6657,1.129408
std,97.328385,126.804446,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,18.808555,0.960933,96.283223,17.245886,0.748421,4.807627,5.765190
std,1.054837,0.078594,2.167803,0.515302,0.021466,0.115496,0.572347


## Baseline

In [ ]:
res = load(open('./results/res.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.45%, E[log Rm]=3.38%, V[log Rm]=8.46%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.935279,33.283870,32.292905,-1.131543
std,40.090256,42.147543,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.465588,-1.056373,39.081091,53.800463,0.522499,10.000180,27.688293
sd,0.115474,0.002290,0.696293,0.321248,0.000781,0.000916,0.822800


## Industries

In [ ]:
res = load(open('./results/res_Health.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.45%, E[log Rm]=3.38%, V[log Rm]=8.46%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.996705,50.746309,50.138498,-1.426878
std,68.962810,75.437620,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.735532,-1.280739,68.106013,15.812377,0.571673,9.999444,9.937278
sd,1.201222,0.184304,3.330918,0.337230,0.004904,0.000812,2.878878


In [ ]:
res = load(open('./results/res_Other.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.30%, E[log Rm]=3.31%, V[log Rm]=8.39%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,16.602002,19.292409,18.041192,-0.766489
std,21.164118,21.729294,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,15.052120,-0.739078,20.235602,86.947560,0.524608,9.992784,67.164687
sd,0.443088,0.049559,7.391515,1.063108,0.002278,0.000824,3.090092


In [ ]:
res = load(open('./results/res_Retail.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.19%, E[log Rm]=3.36%, V[log Rm]=8.64%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,20.528945,29.067261,27.928585,-0.683958
std,38.809052,40.575767,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,19.087538,-0.644586,38.407755,59.136139,0.529450,9.986410,27.906953
sd,0.384297,0.067948,4.335142,1.597876,0.002101,0.000933,2.156463


In [ ]:
res = load(open('./results/res_Tech.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.31%, E[log Rm]=3.36%, V[log Rm]=8.42%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,21.810904,29.105371,28.428608,-1.156679
std,35.452167,37.009672,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.648871,-1.090301,34.243466,61.838131,0.516370,9.998979,31.640361
sd,0.188914,0.020892,1.646535,0.339479,0.000994,0.000193,0.812557


## Before 2000

In [ ]:
res = load(open('./results/res_2000.pkl', 'rb'))
start, end = '1987-01-01', '2000-08-02'
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(res, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=5.26%, E[log Rm]=5.42%, V[log Rm]=7.60%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,101.209273,529.746409,533.417333,-19.798031
std,217.333686,458.100872,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,97.325745,-8.700604,207.043480,9.641510,1.774183,2.101270,3.546142
sd,17.461552,0.597502,19.472328,3.485849,0.413910,0.259416,4.788300


In [ ]:
res

,gamma,delta,sigma,k,a,b,pi_err
value,0.243314,-8.700604,1.035217,0.096415,1.774183,2.101270,0.035461
std,0.043654,0.597502,0.097362,0.361546,0.233296,0.259416,1.399928


# From 2010

## SP500

### Baseline

In [ ]:
res = load(open('../results/res_from_2010.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.044147,38.597624,32.143804,1.369221
std,41.959509,46.519291,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.486135,1.263586,40.741357,58.001268,0.524208,9.976762,25.248838
std,0.561361,0.131019,1.632983,0.542034,0.001035,0.000852,0.753908


### Bootstrap

In [ ]:
bootstrap = load(open('../results/res_from_2010_bootstrap.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,27.915278,38.943901,32.273601,1.418881
std,42.908761,47.645811,0.666767,0.101944


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.177828,1.308279,41.623125,56.730761,0.524225,9.975520,24.313234
std,0.399665,0.054684,1.939227,0.692920,0.001218,0.002243,0.990630


### Industries

In [ ]:
res = load(open('../results/res_from_2010_tech.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.447771,44.398815,36.028574,1.808849
std,48.281191,54.431328,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.350359,1.647008,46.474905,52.761381,0.535861,9.994694,22.433772
std,0.573147,0.096091,2.414756,1.118730,0.001282,0.000661,1.117114


In [ ]:
res = load(open('../results/res_from_2010_other.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.049506,36.410122,33.125152,0.642284
std,48.575001,53.787789,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.171717,0.596017,48.343735,39.852130,0.493348,9.999882,14.341468
std,0.410862,0.011073,2.297246,0.801886,0.001529,0.000870,1.007226


In [ ]:
res = load(open('../results/res_from_2010_health.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,24.738513,28.837675,24.074829,0.98131
std,24.928114,26.829419,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.534031,0.926460,23.817091,85.994521,0.545547,9.999827,54.834035
std,1.589694,0.209106,17.892963,4.103842,0.005613,0.003894,5.782390


In [ ]:
res = load(open('../results/res_from_2010_retail.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.460272,38.650506,30.048132,1.862101
std,38.561903,42.684076,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.079368,1.717618,36.067626,63.138034,0.532096,9.996947,28.519121
std,0.988565,0.081985,6.428220,1.994004,0.003466,0.000788,3.203769


## NASDAQ

### Baseline

In [ ]:
res = load(open('../results/res_from_2010_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^IXIC', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=5.11%, V[log Rm]=9.22%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.921919,39.634118,32.669726,1.272654
std,42.117441,46.808646,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.004577,1.173846,40.703365,58.106907,0.523157,9.994176,25.243450
std,0.858080,0.011246,0.786116,0.572693,0.001619,0.003164,0.789164


### Bootstrap

In [ ]:
bootstrap = load(open('../results/res_from_2010_bootstrap_^IXIC.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.108382,38.802007,32.727919,1.282110
std,42.225155,46.845728,0.513873,0.099973


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.875286,1.182659,41.163643,57.470712,0.523521,9.990994,24.838465
std,0.345110,0.055312,1.856025,0.743290,0.001066,0.000910,1.037502


### Industries

In [ ]:
res = load(open('../results/res_from_2010_tech_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^IXIC', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=5.11%, V[log Rm]=9.22%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,31.297563,41.693922,34.149249,1.386632
std,40.898609,45.637943,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.921910,1.272877,39.178912,63.761897,0.534729,9.999982,28.731251
std,0.364498,0.056970,1.809385,0.569653,0.001500,0.000660,1.204127


In [ ]:
res = load(open('../results/res_from_2010_other_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^IXIC', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=5.11%, V[log Rm]=9.22%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.505009,37.042256,32.563116,0.784501
std,48.828530,54.155284,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,19.649959,0.728229,48.364774,39.894354,0.493309,9.999991,14.368795
std,0.748723,0.154677,2.591675,1.089477,0.001475,0.000882,1.407250


In [ ]:
res = load(open('../results/res_from_2010_health_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^IXIC', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=5.11%, V[log Rm]=9.22%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,25.273573,29.343044,24.715453,0.81366
std,24.670232,26.581027,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,21.230728,0.768808,23.630132,86.440400,0.544876,9.999941,55.002857
std,1.436257,0.197736,15.582735,3.498681,0.005244,0.000741,5.442236


In [ ]:
res = load(open('../results/res_from_2010_retail_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^IXIC', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=5.11%, V[log Rm]=9.22%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.712028,39.845670,31.562703,1.531648
std,38.135860,42.318807,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.693929,1.411696,35.846339,64.148843,0.533119,9.980365,28.557948
std,1.408174,0.229937,8.413139,3.331919,0.003188,0.001673,4.078831


## Russel 2000

### Baseline

In [ ]:
res = load(open('../results/res_from_2010_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^RUT', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=3.44%, V[log Rm]=10.96%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,27.001411,37.692873,33.220907,1.114932
std,42.472941,47.004210,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.475060,1.029217,40.947903,56.874738,0.523091,9.998763,24.609609
std,0.228733,0.036744,1.211804,0.291716,0.000894,0.000725,0.808638


### Bootstrap

In [ ]:
bootstrap = load(open('../results/res_from_2010_bootstrap_^RUT.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.397299,38.711895,33.667923,1.045800
std,41.334481,45.827285,0.578756,0.059216


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.037856,0.965056,40.615566,58.683835,0.524248,9.984860,25.697237
std,0.330391,0.047490,1.943300,0.737501,0.001113,0.001474,0.988238


### Industries

In [ ]:
res = load(open('../results/res_from_2010_tech_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^RUT', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=3.44%, V[log Rm]=10.96%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.758886,39.864857,35.110657,1.19386
std,40.601256,45.111638,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,26.033630,1.096524,38.782192,64.720869,0.536634,9.970507,28.958744
std,0.380943,0.058135,1.995331,0.723783,0.001296,0.001796,1.183004


In [ ]:
res = load(open('../results/res_from_2010_other_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^RUT', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=3.44%, V[log Rm]=10.96%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,22.510357,35.933202,33.168841,0.637394
std,48.784083,53.967209,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.278551,0.591155,48.352004,39.878771,0.493396,10.000003,14.396481
std,0.491275,0.081329,1.921303,0.491491,0.001602,0.000699,1.050168


In [ ]:
res = load(open('../results/res_from_2010_health_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^RUT', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=3.44%, V[log Rm]=10.96%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,24.503158,29.257388,26.131857,0.738397
std,27.326570,29.462730,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,21.963373,0.695372,26.242487,82.616020,0.546694,9.999519,52.414893
std,1.259028,0.157604,7.127250,0.916394,0.005515,0.001235,6.027337


In [ ]:
res = load(open('../results/res_from_2010_retail_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^RUT', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=3.44%, V[log Rm]=10.96%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.539673,38.451266,33.054266,1.373621
std,40.432598,44.776020,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.315431,1.265376,37.980183,59.757663,0.532558,9.997318,27.447623
std,1.169309,0.159583,6.880552,2.717553,0.003813,0.000746,3.600361


# Aggregate

In [ ]:
results = gather_results({ 
    'SP500TR Baseline': '../results/res_from_2010.pkl',
    'SP500TR Tech': '../results/res_from_2010_tech.pkl',
    'SP500TR Retail': '../results/res_from_2010_retail.pkl', 
    'SP500TR Health': '../results/res_from_2010_health.pkl',
    'SP500TR Other': '../results/res_from_2010_other.pkl',
    'NASDAQ Baseline': '../results/res_from_2010_^IXIC.pkl',
    'NASDAQ Tech': '../results/res_from_2010_tech_^IXIC.pkl',
    'NASDAQ Retail': '../results/res_from_2010_retail_^IXIC.pkl', 
    'NASDAQ Health': '../results/res_from_2010_health_^IXIC.pkl',
    'NASDAQ Other': '../results/res_from_2010_other_^IXIC.pkl',
    'Russel Baseline': '../results/res_from_2010_^RUT.pkl',
    'Russel Tech': '../results/res_from_2010_tech_^RUT.pkl',
    'Russel Retail': '../results/res_from_2010_retail_^RUT.pkl', 
    'Russel Health': '../results/res_from_2010_health_^RUT.pkl',
    'Russel Other': '../results/res_from_2010_other_^RUT.pkl',})

results_sp500 = gather_results({ 
    'SP500TR Baseline': '../results/res_from_2010.pkl',
    'SP500TR Tech': '../results/res_from_2010_tech.pkl',
    'SP500TR Retail': '../results/res_from_2010_retail.pkl', 
    'SP500TR Health': '../results/res_from_2010_health.pkl',
    'SP500TR Other': '../results/res_from_2010_other.pkl',})

results_nasdaq = gather_results({ 
    'NASDAQ Baseline': '../results/res_from_2010_^IXIC.pkl',
    'NASDAQ Tech': '../results/res_from_2010_tech_^IXIC.pkl',
    'NASDAQ Retail': '../results/res_from_2010_retail_^IXIC.pkl', 
    'NASDAQ Health': '../results/res_from_2010_health_^IXIC.pkl',
    'NASDAQ Other': '../results/res_from_2010_other_^IXIC.pkl',})

results_russel = gather_results({
    'Russel Baseline': '../results/res_from_2010_^RUT.pkl',
    'Russel Tech': '../results/res_from_2010_tech_^RUT.pkl',
    'Russel Retail': '../results/res_from_2010_retail_^RUT.pkl', 
    'Russel Health': '../results/res_from_2010_health_^RUT.pkl',
    'Russel Other': '../results/res_from_2010_other_^RUT.pkl',})

In [ ]:
results

gamma (%)   delta sigma (%)   k (%)       a       b  \
SP500TR Baseline value     22.49    1.26     40.74   58.00    0.52    9.98   
                 std      (0.56)  (0.13)    (1.63)  (0.54)  (0.00)  (0.00)   
SP500TR Tech     value     23.35    1.65     46.47   52.76    0.54    9.99   
                 std      (0.57)  (0.10)    (2.41)  (1.12)  (0.00)  (0.00)   
SP500TR Retail   value     22.08    1.72     36.07   63.14    0.53   10.00   
                 std      (0.99)  (0.08)    (6.43)  (1.99)  (0.00)  (0.00)   
SP500TR Health   value     20.53    0.93     23.82   85.99    0.55   10.00   
                 std      (1.59)  (0.21)   (17.89)  (4.10)  (0.01)  (0.00)   
SP500TR Other    value     20.17    0.60     48.34   39.85    0.49   10.00   
                 std      (0.41)  (0.01)    (2.30)  (0.80)  (0.00)  (0.00)   
NASDAQ Baseline  value     23.00    1.17     40.70   58.11    0.52    9.99   
                 std      (0.86)  (0.01)    (0.79)  (0.57)  (0.00)  (0.00)   
NASDAQ Tech      value     24.92    1.27     39.18   63.76    0.53   10.00   
                 std      (0.36)  (0.06)    (1.81)  (0.57)  (0.00)  (0.00)   
NASDAQ Retail    value     23.69    1.41     35.85   64.15    0.53    9.98   
                 std      (1.41)  (0.23)    (8.41)  (3.33)  (0.00)  (0.00)   
NASDAQ Health    value     21.23    0.77     23.63   86.44    0.54   10.00   
                 std      (1.44)  (0.20)   (15.58)  (3.50)  (0.01)  (0.00)   
NASDAQ Other     value     19.65    0.73     48.36   39.89    0.49   10.00   
                 std      (0.75)  (0.15)    (2.59)  (1.09)  (0.00)  (0.00)   
Russel Baseline  value     23.48    1.03     40.95   56.87    0.52   10.00   
                 std      (0.23)  (0.04)    (1.21)  (0.29)  (0.00)  (0.00)   
Russel Tech      value     26.03    1.10     38.78   64.72    0.54    9.97   
                 std      (0.38)  (0.06)    (2.00)  (0.72)  (0.00)  (0.00)   
Russel Retail    value     24.32    1.27     37.98   59.76    0.53   10.00   
                 std      (1.17)  (0.16)    (6.88)  (2.72)  (0.00)  (0.00)   
Russel Health    value     21.96    0.70     26.24   82.62    0.55   10.00   
                 std      (1.26)  (0.16)    (7.13)  (0.92)  (0.01)  (0.00)   
Russel Other     value     20.28    0.59     48.35   39.88    0.49   10.00   
                 std      (0.49)  (0.08)    (1.92)  (0.49)  (0.00)  (0.00)   

                        pi (%) E[ln R] (%) E[R] (%) alpha (%)   beta  
SP500TR Baseline value   25.25       28.04    38.60     32.14   1.37  
                 std    (0.75)     (41.96)  (46.52)     (nan)  (nan)  
SP500TR Tech     value   22.43       30.45    44.40     36.03   1.81  
                 std    (1.12)     (48.28)  (54.43)     (nan)  (nan)  
SP500TR Retail   value   28.52       29.46    38.65     30.05   1.86  
                 std    (3.20)     (38.56)  (42.68)     (nan)  (nan)  
SP500TR Health   value   54.83       24.74    28.84     24.07   0.98  
                 std    (5.78)     (24.93)  (26.83)     (nan)  (nan)  
SP500TR Other    value   14.34       23.05    36.41     33.13   0.64  
                 std    (1.01)     (48.58)  (53.79)     (nan)  (nan)  
NASDAQ Baseline  value   25.24       28.20    38.68     32.65   1.27  
                 std    (0.79)     (41.76)  (46.30)     (nan)  (nan)  
NASDAQ Tech      value   28.73       30.52    40.64     34.11   1.39  
                 std    (1.20)     (40.46)  (45.03)     (nan)  (nan)  
NASDAQ Retail    value   28.56       29.85    38.66     31.50   1.53  
                 std    (4.08)     (37.56)  (41.55)     (nan)  (nan)  
NASDAQ Health    value   55.00       24.80    28.77     24.73   0.81  
                 std    (5.44)     (24.41)  (26.26)     (nan)  (nan)  
NASDAQ Other     value   14.37       23.06    36.49     32.58   0.78  
                 std    (1.41)     (48.71)  (53.95)     (nan)  (nan)  
Russel Baseline  value   24.61       28.09    38.56     33.21   1.11  
                 std    (0.81)     (41.76)

In [ ]:
results.to_csv('../results/all_results.csv')

# Security industry

In [ ]:
res = load(open('../results/res_security_from_2010_^SP500TR.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,31.819274,43.398491,36.102137,1.562499
std,43.318507,48.587109,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,25.608555,1.426157,41.811009,58.360694,0.529238,9.999894,21.137507
std,0.854859,0.174251,3.928668,1.760815,0.003048,0.000809,1.580223


In [ ]:
res = load(open('../results/res_security_from_2010_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^IXIC', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=5.11%, V[log Rm]=9.22%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,31.729678,41.843440,34.683067,1.311148
std,40.166522,44.819453,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,25.676175,1.203267,38.604278,63.119751,0.581753,9.166658,23.792308
std,0.782069,0.099288,3.365894,1.129264,0.012710,0.169158,2.390810


In [ ]:
res = load(open('../results/res_security_from_2010_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^RUT', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=3.44%, V[log Rm]=10.96%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.090579,37.136411,32.507962,1.158693
std,38.527191,42.497738,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.441022,1.070908,36.696170,64.698861,0.642974,8.335096,26.016243
std,0.602310,0.086395,3.950424,2.128973,0.014646,0.153622,2.275446


In [8]:
bootstrap = load(open('../results/res_security_from_2010_bootstrap_^SP500TR.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.369822,44.366892,36.480690,1.697810
std,48.743656,55.357206,5.491709,0.430018


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.695796,1.541555,47.157623,49.926242,0.604478,9.066041,19.268465
std,1.249753,0.215634,3.430399,1.063034,0.014899,0.175346,2.177670


In [9]:
bootstrap = load(open('../results/res_security_from_2010_bootstrap_^IXIC.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.204924,42.743893,35.669165,1.511657
std,44.597969,50.186768,3.626997,0.342343


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.184048,1.378873,43.218084,55.186911,0.583030,9.247852,21.425734
std,1.009972,0.159485,3.509901,1.061742,0.015068,0.181287,2.301333


In [10]:
bootstrap = load(open('../results/res_security_from_2010_bootstrap_^RUT.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.739849,41.625243,35.339661,1.330627
std,43.308870,48.606748,4.824475,0.296713


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.368518,1.217089,42.205547,56.649524,0.619162,8.827833,21.964102
std,0.940562,0.126070,3.557447,1.071186,0.013840,0.135885,2.314050


In [14]:
results_security = gather_results({ 
    'SP500TR Baseline': '../results/res_security_from_2010_^SP500TR.pkl',
    'SP500TR Bootstrap': '../results/res_security_from_2010_bootstrap_^SP500TR.pkl',
    'NASDAQ Baseline': '../results/res_security_from_2010_^IXIC.pkl',
    'NASDAQ Bootstrap': '../results/res_security_from_2010_bootstrap_^IXIC.pkl',
    'RUSSELL Baseline': '../results/res_security_from_2010_^RUT.pkl',
    'RUSSELL Bootstrap': '../results/res_security_from_2010_bootstrap_^RUT.pkl',})

In [18]:
results_security.to_csv('../results/security_results.csv')

In [19]:
results_security

gamma (%)   delta sigma (%)   k (%)       a       b  \
SP500TR Baseline  value     25.61    1.43     41.81   58.36    0.53   10.00   
                  std      (0.85)  (0.17)    (3.93)  (1.76)  (0.00)  (0.00)   
SP500TR Bootstrap value     24.18    1.38     43.22   55.19    0.58    9.25   
                  std      (1.01)  (0.16)    (3.51)  (1.06)  (0.02)  (0.18)   
NASDAQ Baseline   value     25.68    1.20     38.60   63.12    0.58    9.17   
                  std      (0.78)  (0.10)    (3.37)  (1.13)  (0.01)  (0.17)   
NASDAQ Bootstrap  value     24.18    1.38     43.22   55.19    0.58    9.25   
                  std      (1.01)  (0.16)    (3.51)  (1.06)  (0.02)  (0.18)   
RUSSELL Baseline  value     24.44    1.07     36.70   64.70    0.64    8.34   
                  std      (0.60)  (0.09)    (3.95)  (2.13)  (0.01)  (0.15)   
RUSSELL Bootstrap value     24.18    1.38     43.22   55.19    0.58    9.25   
                  std      (1.01)  (0.16)    (3.51)  (1.06)  (0.02)  (0.18)   

                         pi (%) E[ln R] (%) E[R] (%) alpha (%)    beta  
SP500TR Baseline  value   21.14       31.82    43.40     36.10    1.56  
                  std    (1.58)     (43.32)  (48.59)     (nan)   (nan)  
SP500TR Bootstrap value   21.43       30.20    42.74     35.67    1.51  
                  std    (2.30)     (44.60)  (50.19)    (3.63)  (0.34)  
NASDAQ Baseline   value   23.79       30.99    40.85     34.66    1.31  
                  std    (2.39)     (39.77)  (44.27)     (nan)   (nan)  
NASDAQ Bootstrap  value   21.43       30.20    42.74     35.67    1.51  
                  std    (2.30)     (44.60)  (50.19)    (3.63)  (0.34)  
RUSSELL Baseline  value   26.02       29.23    38.02     32.48    1.16  
                  std    (2.28)     (37.67)  (41.62)     (nan)   (nan)  
RUSSELL Bootstrap value   21.43       30.20    42.74     35.67    1.51  
                  std    (2.30)     (44.60)  (50.19)    (3.63)  (0.34)